In [98]:
import os
import json
import numpy as np
import pandas as pd

class Experiment1:
    def __init__(self, autosave=True, autosave_dir="."):
        self.autosave = autosave
        self.autosave_dir = autosave_dir

        self.run_id = 0
        self.experiment_id = 0
        self.step = 0
        self.table = pd.DataFrame(columns=["run_id", "experiment_id"])
        
        if autosave:
            os.makedirs(autosave_dir, exist_ok=True)
            self.benchmark_number = len([d for d in os.listdir(self.autosave_dir) if "benchmark" in d])
            self.table_dir = f"{self.autosave_dir}/benchmark_{self.benchmark_number}/tables"
        else:
            self.benchmark_number = None
            self.table_dir = None
    
    def __next__(self):
        self.run_id += 1
        self.step += 1
        
    def __enter__(self, *args, **kwargs):
        self.start(args, kwargs)
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        self.end()

    def start(self, *args, **kwargs):
        cols = ["run_id", "experiment_id", "step"]
        self.table.loc[self.run_id, cols] = [self.run_id, self.experiment_id, self.step]
        
        for k, v in kwargs.items():
            self.table.loc[self.run_id, k] = v

    def end(self):
        self.step = 0
        
        cols = ["run_id", "experiment_id", "step"]
        self.table[cols] = self.table[cols].astype(int)
        

    def insert_dict(self, obj, *args, **kwargs):
        cols = ["run_id", "experiment_id", "step" , "results"]
        self.table.loc[self.run_id, cols] = [self.run_id, self.experiment_id, self.step, json.dumps(obj)]
        
        for k, v in kwargs.items():
            self.table.loc[self.run_id, k] = v
                
    def load(self, load_file=None):
        if self.autosave:
            self.table = pd.read_csv(f"{self.table_dir}/table.csv")
        else:
            self.table = pd.read_csv(load_file)

    def save(self, save_file=None):
        if self.autosave:
            os.makedirs(self.table_dir, exist_ok=True)
            self.table.to_csv(f"{self.table_dir}/table.csv")
        else:
            self.table.to_csv(save_file)
        


class Experiment2(pd.DataFrame):
    # temporary properties
    _internal_names = pd.DataFrame._internal_names + ["autosave", "autosave_dir", "id_names", "run_id", "experiment_id"]
    _internal_names_set = set(_internal_names)

    # normal properties
    _metadata = ["added_property"]

    @property
    def _constructor(self):
        return Experiment2

    def __init__(
        self,
        data=None,
        index=None,
        columns=None,
        dtype=None,
        copy=None,
        size=30,
        run_id=0,
        experiment_id=0,
        autosave=True,
        autosave_dir=".",
    ):
        data = [[np.nan, np.nan]] * size
        self.id_names = ["run_id", "experiment_id"]
        self.run_id = run_id
        self.experiment_id = experiment_id
        super().__init__(data=data, index=None, columns=self.id_names, dtype=None, copy=None)
        self.loc[self.run_id, self.id_names] = [self.run_id, self.experiment_id]
        self.autosave = autosave
        self.autosave_dir = autosave_dir
    
    def start(self):
        pass
    
    def end(self):
        print(super().dropna())
        
    def next(self):
        self.run_id += 1
        self.loc[self.run_id, self.id_names] = [self.run_id, self.experiment_id]
    
    def insert_dict(self, obj):
        for k, v in obj.items():
            if isinstance(v, dict):
                v = json.dumps(v)
            self.loc[self.run_id, k] = v

    def load(self, filepath=None):
        if self.autosave:
            self = pd.read_csv(f"{self.table_dir}/table.csv")
        else:
            self = pd.read_csv(filepath)
        return self

    def save(self, filepath=None):
        if self.autosave:
            os.makedirs(self.table_dir, exist_ok=True)
            self.to_csv(f"{self.table_dir}/table.csv")
        else:
            self.to_csv(filepath)



In [2]:
# Responseのサンプル
response_sample = {
    "type": "Response",
    "version": {"sampleset_schema": "3.1.0"},
    "num_variables": 20,
    "num_rows": 5,
    "sample_data": {
        "type": "array",
        "data": [[123424], [747616], [256132], [358510], [598528]],
        "data_type": "uint32",
        "shape": [5, 1],
        "use_bytes": False,
    },
    "sample_type": "int8",
    "sample_packed": True,
    "vectors": {
        "energy": {
            "type": "array",
            "data": [-207.0, -243.0, -272.0, -264.0, -157.0],
            "data_type": "float64",
            "shape": [5],
            "use_bytes": False,
        },
        "num_occurrences": {
            "type": "array",
            "data": [1, 1, 1, 1, 1],
            "data_type": "int64",
            "shape": [5],
            "use_bytes": False,
        },
    },
    "variable_labels": [
        "_leint_knapsack_constraint[0]",
        "_leint_knapsack_constraint[1]",
        "_leint_knapsack_constraint[2]",
        "_leint_knapsack_constraint[3]",
        "_leint_knapsack_constraint[4]",
        "_leint_knapsack_constraint[5]",
        "_leint_knapsack_constraint[6]",
        "_leint_knapsack_constraint[7]",
        "_leint_knapsack_constraint[8]",
        "_leint_knapsack_constraint[9]",
        "x[0]",
        "x[1]",
        "x[2]",
        "x[3]",
        "x[4]",
        "x[5]",
        "x[6]",
        "x[7]",
        "x[8]",
        "x[9]",
    ],
    "variable_type": "BINARY",
    "info": {
        "system": [],
        "sampling_time": 4100.264981389046,
        "execution_time": 751.1120289564133,
        "list_exec_times": {
            "type": "array",
            "data": [
                858.2137525081635,
                771.5118117630482,
                698.3112543821335,
                719.3121127784252,
                708.211213350296,
            ],
            "data_type": "float64",
            "shape": [5],
            "use_bytes": False,
        },
        "schedule": {
            "beta_max": 4.605170185988092,
            "beta_min": 4.652244285330389e-06,
            "num_sweeps": 1000,
        },
    },
}

In [244]:
static_params = {
    "num_sweeps": [10, 20, 30],
    "num_reads": [1, 2, 3]
}
steps = range(3)

experiment = Experiment1()
run_id = 0
experiment_id = 0
for num_sweep in static_params["num_sweeps"]:
    for num_reads in static_params["num_reads"]:
        args = {"num_sweeps": num_sweep, "num_reads": num_reads}
        experiment.start(**args)
        for step in steps:
            # ここではsolve関数は上のresponse_sampleを返す想定
            # response_sample = solve()
            results = {"response": response_sample}
            experiment.insert_dict(results, **args)
            
            next(experiment)
        experiment.end()


In [193]:
experiment.table

,run_id,experiment_id,step,num_sweeps,num_reads,results
0,0,0,0,10.0,1.0,"{""response"": {""type"": ""Response"", ""version"": {..."
1,1,0,1,10.0,1.0,"{""response"": {""type"": ""Response"", ""version"": {..."
2,2,0,2,10.0,1.0,"{""response"": {""type"": ""Response"", ""version"": {..."
3,3,0,0,10.0,2.0,"{""response"": {""type"": ""Response"", ""version"": {..."
4,4,0,1,10.0,2.0,"{""response"": {""type"": ""Response"", ""version"": {..."
5,5,0,2,10.0,2.0,"{""response"": {""type"": ""Response"", ""version"": {..."
6,6,0,0,10.0,3.0,"{""response"": {""type"": ""Response"", ""version"": {..."
7,7,0,1,10.0,3.0,"{""response"": {""type"": ""Response"", ""version"": {..."
8,8,0,2,10.0,3.0,"{""response"": {""type"": ""Response"", ""version"": {..."
9,9,0,0,20.0,1.0,"{""response"": {""type"": ""Response"", ""version"": {..."


In [189]:
experiment.save()

In [202]:

with Experiment1() as experiment:
    print(experiment)


In [103]:
experiment = Experiment2()
static_params = {
    "num_sweeps": [10, 20, 30],
    "num_reads": [1, 2, 3]
}
steps = range(3)

experiment = Experiment2()
run_id = 0
experiment_id = 0
for num_sweep in static_params["num_sweeps"]:
    for num_reads in static_params["num_reads"]:
        args = {"num_sweeps": num_sweep, "num_reads": num_reads}
        experiment.start()
        for step in steps:
            # ここではsolve関数は上のresponse_sampleを返す想定
            # response_sample = solve()
            args["step"] = step
            args["response"] = response_sample
            experiment.insert_dict(args)
            experiment.next()
        print(experiment.dtypes)
        print(experiment.response.dtype)
        experiment.end()



run_id           float64
experiment_id    float64
num_sweeps       float64
num_reads        float64
step             float64
response          object
dtype: object
object


TypeError: ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [97]:
experiment

,run_id,experiment_id,num_sweeps,num_reads,step,response
0,0.0,0.0,10.0,1.0,0.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
1,1.0,0.0,10.0,1.0,1.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
2,2.0,0.0,10.0,1.0,2.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
3,3.0,0.0,10.0,2.0,0.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
4,4.0,0.0,10.0,2.0,1.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
5,5.0,0.0,10.0,2.0,2.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
6,6.0,0.0,10.0,3.0,0.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
7,7.0,0.0,10.0,3.0,1.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
8,8.0,0.0,10.0,3.0,2.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
9,9.0,0.0,20.0,1.0,0.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."


In [339]:
experiment = Experiment2()
# sample_data = {"a": 1, "response": {"version": "0.1"}}
experiment.loc[0, "a"] = 1
# experiment.insert_dict(sample_data)
experiment.loc[1, "a"] = 2
experiment


RecursionError: maximum recursion depth exceeded in __instancecheck__

In [358]:
experiment = pd.DataFrame([[0]], columns=["a"])

data = {'run_id': 1, 'experiment_id': 0}
experiment = experiment.assign(**data, index=[1])
experiment


,a,run_id,experiment_id,index
0,0,1,0,1


In [278]:
df = pd.DataFrame()

df.insert(0, "a", [{"aa": 1}])
df.insert(0, "b", [5])
print(df)
print()
print(df.iloc[-1])

   b          a
0  5  {'aa': 1}

b            5
a    {'aa': 1}
Name: 0, dtype: object
